# CODE TO CREATE TRAIN AND VERIFICATION TEST DATA 

In [61]:
import pandas as pd
import os

In [62]:
master_df = pd.read_csv("train dataset.csv", encoding = "ISO-8859-1")
master_df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [63]:
master_df_test = pd.DataFrame(master_df).iloc[5000:].reset_index(drop = True, inplace = False).rename(columns={"Category":"label", "Message":"message"})
master_df_train = pd.DataFrame(master_df).iloc[0:5000].reset_index(drop = True, inplace = False).rename(columns={"Category":"label", "Message":"message"})


if not os.path.exists("test"):
    os.makedirs("test")

for index, row in master_df_test.iterrows():

    
    row_data = str(row[1])  # Assuming the column is unnamed or you want the first column
    
    # Define a filename for each text file
    filename = os.path.join("test", f"email{index+1}.txt")
    
    # Write the row data to a text file
    with open(filename, "w", encoding = "ISO-8859-1") as f:
        f.write(row_data)

C:\Users\sakth\AppData\Local\Temp\ipykernel_14572\3610428285.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_data = str(row[1])  # Assuming the column is unnamed or you want the first column


In [64]:
master_df_test

,label,message
0,ham,"Hmph. Go head, big baller."
1,ham,Well its not like you actually called someone ...
2,ham,"Nope. Since ayo travelled, he has forgotten hi..."
3,ham,You still around? Looking to pick up later
4,spam,CDs 4u: Congratulations ur awarded Â£500 of CD...
...,...,...
567,spam,This is the 2nd time we have tried 2 contact u...
568,ham,Will Ã¼ b going to esplanade fr home?
569,ham,"Pity, * was in mood for that. So...any other s..."
570,ham,The guy did some bitching but I acted like i'd...


In [65]:
master_y_true = master_df_test["label"]
master_y_true.to_csv("y_true.csv", index=False)

In [66]:
dataset = master_df_train 

# ACTUAL CODE 

In [67]:
import re
import numpy as np
from collections import Counter

In [68]:
# Preprocess text data
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters and punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [69]:
dataset['message'] = dataset['message'].apply(preprocess_text)
X_train = dataset['message'].values

# Convert labels to binary format (1 for spam, 0 for ham)
dataset['label'] = dataset['label'].map({'spam': 1, 'ham': 0})
y_train = dataset['label'].values

In [70]:
X_train

array(['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat',
       'ok lar joking wif u oni',
       'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question std txt rate t c s apply 08452810075over18 s',
       ..., 'happy new year hope you are having a good semester',
       'esplanade lor where else', 'can you talk with me'], dtype=object)

In [71]:
y_train

array([0, 0, 1, ..., 0, 0, 0])

In [72]:
# Manual Feature Extraction using Bag of Words
def build_vocabulary(messages, vocab_size=1000):
    # Tokenize and count word frequencies across all messages
    word_counts = Counter()
    for message in messages:
        words = message.split()
        word_counts.update(words)
    
    # Select the top `vocab_size` words as the vocabulary
    vocabulary = [word for word, _ in word_counts.most_common(vocab_size)]
    return vocabulary

def message_to_vector(message, vocabulary):
    words = message.split()
    word_count = Counter(words)
    # Create a vector of size `vocab_size` where each position represents a word count in the vocabulary
    vector = [word_count[word] if word in word_count else 0 for word in vocabulary]
    return vector

In [73]:
# Separate the dataset into spam and ham
spam_data = dataset[dataset['label'] == 1]
ham_data = dataset[dataset['label'] == 0]

print("Number of spam messages:", len(spam_data))
print("Number of ham messages:", len(ham_data))

Number of spam messages: 673
Number of ham messages: 4327


In [74]:
# Oversample spam data to match the size of ham data
oversampled_spam_data = spam_data.sample(len(ham_data), replace=True)
balanced_data = pd.concat([ham_data, oversampled_spam_data])

# Shuffle the balanced dataset  
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract messages and labels
X_train = balanced_data['message']
y_train = balanced_data['label'].values


# # Rebuild the vocabulary and feature matrix for the balanced data
# vocabulary = build_vocabulary(X)
# X_balanced = np.array([message_to_vector(message, vocabulary) for message in X])

In [75]:
folder_path = "test" 
data = []

if not os.path.exists("test"):
    print("test folder does not exist")
    
if not os.path.exists(folder_path):
    print(f"{folder_path} folder does not exist")
else:
    # Sort filenames numerically by extracting the numeric part
    filenames = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0].replace('email', '')))
    
    # Loop through each sorted file
    for filename in filenames:
        if filename.endswith(".txt"):  # Check for .txt files
            file_path = os.path.join(folder_path, filename)  # Full path to the file
            
            # Read the file content
            with open(file_path, "r", encoding="ISO-8859-1") as file:
                content = file.read().strip()  # Read and strip whitespace/newlines
                
            # Append the content to the list
            data.append({"message": content})

# Create a DataFrame from the list
master_df_test = pd.DataFrame(data)

In [76]:
master_df_test

,message
0,"Hmph. Go head, big baller."
1,Well its not like you actually called someone ...
2,"Nope. Since ayo travelled, he has forgotten hi..."
3,You still around? Looking to pick up later
4,CDs 4u: Congratulations ur awarded Â£500 of CD...
...,...
567,This is the 2nd time we have tried 2 contact u...
568,Will Ã¼ b going to esplanade fr home?
569,"Pity, * was in mood for that. So...any other s..."
570,The guy did some bitching but I acted like i'd...


In [77]:
# Extract messages and labels
X_test = master_df_test['message'].values

# Preprocess text data
X_test = [preprocess_text(text) for text in X_test]


In [78]:
def accuracy_score(y_true, y_pred):
    pred = np.sum(y_true == y_pred)
    acc = pred / len(y_true)
    return acc

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 2))
X_train =vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

y_test = pd.read_csv("y_true.csv", header=None, skiprows=1)
y_test = y_test[0].map({'spam': 1, 'ham': 0})
y_test=y_test.values

In [80]:
X_train.shape

(8654, 1000)

In [81]:
# Naive Bayes Classifier from Scratch
class NaiveBayesClassifier:
    def __init__(self):
        self.word_probs = {}
        self.class_probs = {}

    def fit(self, X, y):
        # Calculate prior probabilities
        self.class_probs[1] = np.mean(y == 1)  # Probability of spam
        self.class_probs[0] = np.mean(y == 0)  # Probability of ham
        
        # Separate spam and ham messages
        spam_messages = X[y == 1]
        ham_messages = X[y == 0]
        
        # Calculate total word counts for each class
        total_spam_words = np.sum(spam_messages)
        total_ham_words = np.sum(ham_messages)
        
        # Calculate word probabilities given each class
        self.word_probs[1] = (np.sum(spam_messages, axis=0) + 1) / (total_spam_words + X.shape[1])
        self.word_probs[0] = (np.sum(ham_messages, axis=0) + 1) / (total_ham_words + X.shape[1])

    def predict(self, X):
        predictions = []
        for message in X:
            # Calculate log-probabilities for spam and ham
            log_spam_prob = np.log(self.class_probs[1])
            log_ham_prob = np.log(self.class_probs[0])
            for i, word_count in enumerate(message):
                if word_count > 0:  # Only consider words that appear in the message
                    log_spam_prob += word_count * np.log(self.word_probs[1][i])
                    log_ham_prob += word_count * np.log(self.word_probs[0][i])
            # Choose the class with higher log-probability
            predictions.append(1 if log_spam_prob > log_ham_prob else 0)
        return np.array(predictions)

In [82]:


from sklearn.metrics import classification_report
# Initialize and train the Naive Bayes classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = nb_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy_nb = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy_nb}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9772727272727273
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       498
           1       0.89      0.95      0.92        74

    accuracy                           0.98       572
   macro avg       0.94      0.96      0.95       572
weighted avg       0.98      0.98      0.98       572



In [83]:
from sklearn.svm import SVC
svc_classifier = SVC()
svc_classifier.fit(X_train, y_train)
y_pred = svc_classifier.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_svc}")
print(classification_report(y_test, y_pred))    

Accuracy: 0.9842657342657343
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       498
           1       0.96      0.92      0.94        74

    accuracy                           0.98       572
   macro avg       0.97      0.96      0.96       572
weighted avg       0.98      0.98      0.98       572



In [ ]:
class KNearestNeighborsClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))  # Calculate Euclidean distance
            k_nearest_indices = distances.argsort()[:self.k]  # Get indices of k nearest neighbors
            k_nearest_labels = self.y_train[k_nearest_indices]  # Get the labels of those neighbors
            majority_vote = Counter(k_nearest_labels).most_common(1)[0][0]  # Majority vote
            predictions.append(majority_vote)
        return np.array(predictions)


In [85]:
# Initialize and train the K-Nearest Neighbors classifier
knn_classifier = KNearestNeighborsClassifier(k=3)
knn_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_knn = knn_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print("\nK-Nearest Neighbors (K=3) Performance:")
print(f"Accuracy: {accuracy_knn}")
print(classification_report(y_test, y_pred_knn))



K-Nearest Neighbors (K=3) Performance:
Accuracy: 0.9632867132867133
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       498
           1       0.91      0.80      0.85        74

    accuracy                           0.96       572
   macro avg       0.94      0.89      0.91       572
weighted avg       0.96      0.96      0.96       572



In [86]:
class DecisionTreeClassifierCustom:
    def __init__(self, depth=10):
        self.depth = depth

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=self.depth)

    def _build_tree(self, X, y, depth):
        # Base cases for stopping recursion
        if len(set(y)) == 1 or depth == 0:
            return Counter(y).most_common(1)[0][0]
        
        # Find best split
        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            return Counter(y).most_common(1)[0][0]
        
        # Split dataset
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold
        
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth-1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth-1)
        
        return (best_feature, best_threshold, left_subtree, right_subtree)

    def _best_split(self, X, y):
        best_gain = 0
        best_feature, best_threshold = None, None
        # Loop over each feature to find the best split
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self._information_gain(y, X[:, feature], threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def _information_gain(self, y, feature_column, threshold):
        # Calculate information gain for a specific split
        left = y[feature_column <= threshold]
        right = y[feature_column > threshold]
        if len(left) == 0 or len(right) == 0:
            return 0
        p = len(left) / len(y)
        return self._entropy(y) - (p * self._entropy(left) + (1 - p) * self._entropy(right))

    def _entropy(self, y):
        # Calculate entropy
        proportions = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in proportions if p > 0])

    def _predict(self, node, x):
        # Recursively traverse the tree
        if not isinstance(node, tuple):
            return node
        feature, threshold, left, right = node
        if x[feature] <= threshold:
            return self._predict(left, x)
        else:
            return self._predict(right, x)

    def predict(self, X):
        return np.array([self._predict(self.tree, x) for x in X])

In [87]:
# Initialize and train the Decision Tree classifier
tree_classifier = DecisionTreeClassifierCustom(depth=5)
tree_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_tree = tree_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy_tree = accuracy_score(y_test, y_pred_tree)

print("\nDecision Tree Performance:")
print(f"Accuracy: {accuracy_tree}")
print(classification_report(y_test, y_pred_tree))


Decision Tree Performance:
Accuracy: 0.9300699300699301
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       498
           1       0.80      0.61      0.69        74

    accuracy                           0.93       572
   macro avg       0.87      0.79      0.83       572
weighted avg       0.93      0.93      0.93       572



In [88]:
class LogisticRegressionClassifier:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        for _ in range(self.epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)
            # Gradient descent updates
            dw = (1 / len(y)) * np.dot(X.T, (y_pred - y))
            db = (1 / len(y)) * np.sum(y_pred - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        return [1 if i > 0.5 else 0 for i in y_pred]

In [89]:
# Initialize and train the Logistic Regression classifier
logistic_classifier = LogisticRegressionClassifier(lr=0.01, epochs=1000)
logistic_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_logistic = logistic_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)

print("\nLogistic Regression Performance:")
print(f"Accuracy: {accuracy_logistic}")
print(classification_report(y_test, y_pred_logistic))


Logistic Regression Performance:
Accuracy: 0.972027972027972
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       498
           1       0.84      0.97      0.90        74

    accuracy                           0.97       572
   macro avg       0.92      0.97      0.94       572
weighted avg       0.98      0.97      0.97       572



In [ ]:
y_test = np.array(y_test)
unique_elements, counts = np.unique(y_test, return_counts=True)
frequency_dict = dict(zip(unique_elements, counts))

print("Unique elements:", unique_elements)
print("Counts:", counts)
print("Frequency dictionary:", frequency_dict)

Unique elements: [0 1]
Counts: [498  74]
Frequency dictionary: {np.int64(0): np.int64(498), np.int64(1): np.int64(74)}


: 